In [1]:
import os
import requests
import pandas as pd
import sqlite3

- Download dos arquivos

In [11]:
# URL para os datasets do INDB
url_base = "https://datasets.imdbws.com/"

# Lista dos arquivos a baixar
arquivos = [
    "name.basics.tsv.gz",
    "title.akas.tsv.gz",
    "title.basics.tsv.gz",
    "title.crew.tsv.gz",
    "title.episode.tsv.gz",
    "title.principals.tsv.gz",
    "title.ratings.tsv.gz"
]

destino_diretorio = 'data'

# verificando a existencia do diretorio 'data'
os.makedirs( destino_diretorio, exist_ok=True )

# Loop para baixar os arquivos
for arquivo in arquivos:
    url = url_base + arquivo
    caminho_destino = os.path.join( destino_diretorio, arquivo )

    # verificando se o arquivo já existe no diretorio 'data'
    if not os.path.exists( caminho_destino ):
        print( f'Baixando {arquivo}...' )
        response = requests.get( url )

        # verificando solicitação ( código status = 200 )
        if response.status_code == 200:
            with open( caminho_destino, 'wb' ) as f:
                f.write( response.content )
            print(f'{arquivo} bixado com sucesso!')
        
        else:
            print(f' Falaha ao baixar {arquivo}. Código de status: {response.status_code}')

    else:
        print(f'{arquivo} já esxiste. Pulando Downaload')
print( 'Download Concluído!!')


name.basics.tsv.gz já esxiste. Pulando Downaload
title.akas.tsv.gz já esxiste. Pulando Downaload
title.basics.tsv.gz já esxiste. Pulando Downaload
title.crew.tsv.gz já esxiste. Pulando Downaload
title.episode.tsv.gz já esxiste. Pulando Downaload
title.principals.tsv.gz já esxiste. Pulando Downaload
title.ratings.tsv.gz já esxiste. Pulando Downaload
Download Concluído!!


- Tratamento dos dados

In [2]:

# Diretórios
diretorio_dados = 'data'
diretorio_tratados = os.path.join(diretorio_dados, 'tratados')

# Verificando se o diretório 'tratados' já existe
os.makedirs(diretorio_tratados, exist_ok=True)

# Lista todos os arquivos no diretório 'data'
arquivos = os.listdir(diretorio_dados)

# Função para processar chunks
def processar_chunk(chunk, caminho_destino):
    # Substituindo os caracteres '\N' por None
    chunk.replace({'\\N': None}, inplace=True)
    # Salvando o dataframe no diretório 'tratados' sem compressão
    chunk.to_csv(caminho_destino, sep='\t', index=False, mode='a', header=not os.path.exists(caminho_destino))

# Loop para abrir, tratar e salvar o arquivo
for arquivo in arquivos:
    caminho_arquivo = os.path.join(diretorio_dados, arquivo)

    if not os.path.isfile(caminho_arquivo) and arquivo.endswith('.gz'):
        print(f'Lendo e tratando o arquivo {arquivo}...')

        # Definindo o caminho de destino para salvar o arquivo tratado
        caminho_destino = os.path.join(diretorio_tratados, arquivo[:-3])  # removendo a extensão .gz

        # Lê o arquivo TSV usando pandas em chunks
        try:
            chunksize = 10000  # Tamanho do chunk, ajuste conforme necessário
            for chunk in pd.read_csv(caminho_arquivo, sep='\t', compression='gzip', low_memory=False, chunksize=chunksize):
                processar_chunk(chunk, caminho_destino)
        except Exception as e:
            print(f'Erro ao processar o arquivo {arquivo}: {e}')
            continue

        print(f'Tratamento concluído para {arquivo}. Salvo em {caminho_destino}')
    else:
        print(f'Aquivo {arquivo} já está tratado.')

print('Todos os arquivos foram tratados e salvos no diretório tratados')


Aquivo name.basics.tsv.gz já está tratado.
Aquivo title.akas.tsv.gz já está tratado.
Aquivo title.basics.tsv.gz já está tratado.
Aquivo title.crew.tsv.gz já está tratado.
Aquivo title.episode.tsv.gz já está tratado.
Aquivo title.principals.tsv.gz já está tratado.
Aquivo title.ratings.tsv.gz já está tratado.
Aquivo tratados já está tratado.
Todos os arquivos foram tratados e salvos no diretório tratados
